In [1]:
import pts
from pts.modules import MeanScaler
from pts.model import weighted_average
from pts.model.time_grad.epsilon_theta import DiffusionEmbedding
from pts.model.time_grad import TimeGradTrainingNetwork, TimeGradPredictionNetwork

import argparse
import numpy as np
import torch
from copy import deepcopy

from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

c:\Users\cesar\anaconda3\envs\diffusion-flow\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pts
from pts.modules import MeanScaler
from pts.model import weighted_average
from pts.model.time_grad.epsilon_theta import DiffusionEmbedding
from pts.model.time_grad import TimeGradTrainingNetwork, TimeGradPredictionNetwork

import argparse
import numpy as np
import torch
from copy import deepcopy

from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

from gluonts.transform import (
    Transformation,
    Chain,
    InstanceSplitter,
    ExpectedNumInstanceSampler,
    ValidationSplitSampler,
    TestSplitSampler,
    RenameFields,
    AsNumpyArray,
    ExpandDimArray,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    VstackFeatures,
    SetFieldIfNotPresent,
    TargetDimIndicator,
)
from gluonts.dataset.field_names import FieldName

from pts.feature import (
    fourier_time_features_from_frequency,
    lags_for_fourier_time_features_from_frequency,
)


# Set Dataset

In [3]:
dataset="electricity_nips"

# Load data
dataset = get_dataset(dataset, regenerate=False)

target_dim = int(dataset.metadata.feat_static_cat[0].cardinality)

train_grouper = MultivariateGrouper(max_target_dim=min(2000, target_dim))
test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test) / len(dataset.train)), max_target_dim=min(2000, target_dim))
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

val_window = 20 * dataset.metadata.prediction_length
dataset_train = list(dataset_train)
dataset_val = []
for i in range(len(dataset_train)):
    x = deepcopy(dataset_train[i])
    x['target'] = x['target'][:,-val_window:]
    dataset_val.append(x)
    dataset_train[i]['target'] = dataset_train[i]['target'][:,:-val_window]

In [4]:
dataset_train[0]['target'].shape

(370, 5353)

# Define Transforms

In [5]:
freq = dataset.metadata.freq
prediction_length=dataset.metadata.prediction_length

time_features = None
lags_seq = None

time_features = (
    time_features
    if time_features is not None
    else fourier_time_features_from_frequency(freq)
)

lags_seq = (
    lags_seq
    if lags_seq is not None
    else lags_for_fourier_time_features_from_frequency(freq_str=freq)
)


transformations = Chain(
            [
                AsNumpyArray(
                    field=FieldName.TARGET,
                    expected_ndim=1,
                ),
                # maps the target to (1, T)
                # if the target data is uni dimensional
                ExpandDimArray(
                    field=FieldName.TARGET,
                    axis=None,
                ),
                AddObservedValuesIndicator(
                    target_field=FieldName.TARGET,
                    output_field=FieldName.OBSERVED_VALUES,
                ),
                AddTimeFeatures(
                    start_field=FieldName.START,
                    target_field=FieldName.TARGET,
                    output_field=FieldName.FEAT_TIME,
                    time_features=time_features,
                    pred_length=prediction_length,
                ),
                VstackFeatures(
                    output_field=FieldName.FEAT_TIME,
                    input_fields=[FieldName.FEAT_TIME],
                ),
                SetFieldIfNotPresent(field=FieldName.FEAT_STATIC_CAT, value=[0]),
                TargetDimIndicator(
                    field_name="target_dimension_indicator",
                    target_field=FieldName.TARGET,
                ),
                AsNumpyArray(field=FieldName.FEAT_STATIC_CAT, expected_ndim=1),
            ]
        )

In [6]:
from gluonts.dataset.loader import TrainDataLoader
from gluonts.itertools import Cached
from gluonts.torch.batchify import batchify

In [11]:
from pts.dataset.loader import TransformedIterableDataset

In [7]:
batch_size = 32
num_batches_per_epoch = 50

In [ ]:
def create_instance_splitter(self, mode: str):
    assert mode in ["training", "validation", "test"]

    instance_sampler = {
        "training": self.train_sampler,
        "validation": self.validation_sampler,
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field=FieldName.TARGET,
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=self.history_length,
        future_length=self.prediction_length,
        time_series_fields=[
            FieldName.FEAT_TIME,
            FieldName.OBSERVED_VALUES,
        ],
    ) + (
        RenameFields(
            {
                f"past_{FieldName.TARGET}": f"past_{FieldName.TARGET}_cdf",
                f"future_{FieldName.TARGET}": f"future_{FieldName.TARGET}_cdf",
            }
        )
    )

In [ ]:
training_iter_dataset = TransformedIterableDataset(
    dataset=training_data,
    transform=transformation
    + training_instance_splitter
    + SelectFields(input_names),
    is_train=True,
    shuffle_buffer_length=shuffle_buffer_length,
    cache_data=cache_data,
)

In [ ]:
data_loader = TrainDataLoader(
    # We cache the dataset, to make training faster
    Cached(dataset.train),
    batch_size=batch_size,
    stack_fn=batchify,
    transform=transformations,
    num_batches_per_epoch=num_batches_per_epoch,
)

TypeError: TrainDataLoader() missing 1 required keyword-only argument: 'stack_fn'

In [9]:
next(data_loader.__iter__())

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (32,) + inhomogeneous part.